# Create organized IMDB dataset

In this notebook, we will clean up the bigger IMDB dataset and create a organized csv file for future use

## Loading the Library

In [6]:
import pandas as pd
import os

## Loading the Data

The following code loads the dataset into a DataFrame and displays the first few rows.

In [2]:
#load the data, get rid of unnecessary columns, and remove the 'tt' from the tconst and parentTconst columns
df = pd.read_csv('../data/merged_imdb_data.csv') #load the data

df = df.drop(columns=['endYear', 'originalTitle','ordering', 'region', 'attributes', 'isOriginalTitle', 'language', 'types', 'titleType']) #drop unnecessary columns

df = df.rename(columns={
    'tconst': 'episode_id',
    'parentTconst': 'show_id',
    'seasonNumber': 'season',
    'episodeNumber': 'episode',
    'averageRating': 'rating',
    'numVotes': 'votes',
    'startYear': 'year',
    'title': 'show_title',
    'runtimeMinutes': 'runtime_minutes',
    'primaryTitle': 'episode_title',
    'genres': 'genre',
    'isAdult': 'is_adult',
})

print(df.shape) #print the shape of the dataframe
df.head() #`print the first 5 rows of the dataframe

/var/folders/rm/dnyhsnfd6ll4tl6sg03cj7t80000gn/T/ipykernel_71950/166840253.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/merged_imdb_data.csv') #load the data


(718795, 12)


,episode_id,show_id,season,episode,rating,votes,episode_title,is_adult,year,runtime_minutes,genre,show_title
0,tt0041951,tt0041038,1,9,7.6,90,The Tenderfeet,0,1949,30,Western,The Lone Ranger
1,tt0635319,tt0041038,4,13,8.4,44,A Broken Match,0,1954,23,Western,The Lone Ranger
2,tt0635320,tt0041038,5,21,8.9,47,A Harp for Hannah,0,1957,23,Western,The Lone Ranger
3,tt0635321,tt0041038,1,41,7.7,63,A Pardon for Curley,0,1950,23,Western,The Lone Ranger
4,tt0635322,tt0041038,3,27,7.6,37,A Stage for Mademoiselle,0,1953,23,Western,The Lone Ranger


### Checking missing or wrong values

In [ ]:
print(df['is_adult'].value_counts())


# Get rows where 'isAdult' column has values different from 0 or 1
df_invalid_isAdult = df[~df['is_adult'].isin([0, 1])]
df_invalid_isAdult.head()

## Data Cleaning

Next, we will clean the data by handling missing values and removing duplicates.

In [3]:
df_new = df.copy()

# Convert the data types of the columns to the correct data type
df_new['episode_id'] = df_new['episode_id'].astype(str)
df_new['show_id'] = df_new['show_id'].astype(str)
df_new['episode_title'] = df_new['episode_title'].astype(str)  
df_new['show_title'] = df_new['show_title'].astype(str)

# Convert to numeric and handle NaN values
df_new['season'] = pd.to_numeric(df_new['season'], errors='coerce').fillna(pd.NA).astype('Int64')
df_new['episode'] = pd.to_numeric(df_new['episode'], errors='coerce').fillna(pd.NA).astype('Int64')
df_new['votes'] = pd.to_numeric(df_new['votes'], errors='coerce').fillna(pd.NA).astype('Int64')
df_new['runtime_minutes'] = pd.to_numeric(df_new['runtime_minutes'], errors='coerce').fillna(pd.NA).astype('Int64')
df_new['rating'] = pd.to_numeric(df_new['rating'], errors='coerce').fillna(pd.NA).astype(float)
df_new['year'] = pd.to_numeric(df_new['year'], errors='coerce').fillna(pd.NA).astype('Int64')

# Handle non-boolean values in 'is_adult' and convert to boolean
df_new.loc[~df_new['is_adult'].isin([0, 1]), 'is_adult'] = pd.NA
df_new['is_adult'] = df_new['is_adult'].astype('boolean')

df_new.dtypes

episode_id          object
show_id             object
season               Int64
episode              Int64
rating             float64
votes                Int64
episode_title       object
is_adult           boolean
year                 Int64
runtime_minutes      Int64
genre               object
show_title          object
dtype: object

In [ ]:
print(df_new['is_adult'].unique())

# Get rows where 'isAdult' column has values different from 0 or 1
df_invalid_isAdult = df_new[~df_new['is_adult'].isin([0, 1])]
df_invalid_isAdult.head()

## Final Summary

The column names and the values of the first row

In [4]:
# Extract the first row
first_row = df_new.iloc[0]

# Convert the first row to a new DataFrame
summary_df = pd.DataFrame(first_row).reset_index()
summary_df.columns = ['Column Name', 'First Row Value']

print(summary_df)
del summary_df

        Column Name  First Row Value
0        episode_id        tt0041951
1           show_id        tt0041038
2            season                1
3           episode                9
4            rating              7.6
5             votes               90
6     episode_title   The Tenderfeet
7          is_adult            False
8              year             1949
9   runtime_minutes               30
10            genre          Western
11       show_title  The Lone Ranger


## Export

In [7]:
filename = 'organized_imdb_dataset.csv'

# Construct the path to the data folder outside the notebooks folder
data_folder = os.path.join('..', 'data')
os.makedirs(data_folder, exist_ok=True)

# Export tvshow_df as a CSV file into the data folder
file_path = os.path.join(data_folder, filename)
df_new.to_csv(file_path, index=False)

print(f"File saved to {file_path}")

File saved to ../data/organized_imdb_dataset.csv
